In [ ]:
import sys, os
import random

sys.path.append("..")
from common_utils import DATA_HOME
import torch
import numpy as np

seed = 42

g = torch.Generator()
g.manual_seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)

from lstm.sales_data import Sales_Dataset
from torch.utils.data.dataloader import DataLoader

In [ ]:
SALE_HOME = os.path.join(DATA_HOME, "sales_data")

I, H, B = 71, 34, 10
HEAD = 6
SEQ_LEN = 716

sd = Sales_Dataset(SALE_HOME, seq_len=SEQ_LEN)
test_set, train_set = torch.utils.data.random_split(sd, [0.1, 0.9])
train_dl = DataLoader(train_set, shuffle=True, batch_size=B, generator=g)
test_dl = DataLoader(test_set, shuffle=True, batch_size=B, generator=g)
len(train_dl), len(test_dl)

In [ ]:
print(len(train_dl), len(test_dl))
# print(sd.O.describe())
# print(sd.TS.describe())
# print(sd.S.describe())
print(sd[0][0].shape, sd[4][0].shape)
# print(sd[4][1].shape)
# print(sd[4][0], sd[4][1])

### PyTorch Model training

In [ ]:
from torch.nn import LSTM
from torch.nn import MSELoss
from torch import optim
import matplotlib.pyplot as plt
from IPython.display import clear_output

model = LSTM(I, H, batch_first=True).cuda()

saved_iter = 500
resume_from = f"sales_model_{saved_iter}_bs{B}.pth"
if os.path.exists(resume_from):
    model.load_state_dict(torch.load(resume_from))
    train_gen = enumerate(train_dl, saved_iter)
else:
    train_gen = enumerate(train_dl)
test_gen = enumerate(test_dl)

loss = MSELoss()
adam = optim.Adam(model.parameters(), lr=0.001)
tr_losses, inf_losses = [], []


fig = plt.figure(figsize=(5, 5))
for tr_idx, (X, y) in train_gen:
    X = X.cuda()
    y = y.cuda()

    # Train
    tr_l, inf_l = 0, 0
    yhat, (_, _) = model(X)
    l = loss(yhat[:, -16:], y)
    # update parameters
    adam.zero_grad()
    l.backward()
    adam.step()
    tr_l += l.item()

    # Validate
    try:
        inf_idx, (X, y) = next(test_gen)
    except StopIteration:  # re-initialize test set if exhausted
        test_gen = enumerate(test_dl)
        inf_idx, (X, y) = next(test_gen)

    X = X.cuda()
    y = y.cuda()
    yhat, (_, _) = model(X)
    l = loss(yhat[:, -16:], y)
    inf_l += l.item()
    tr_losses.append(tr_l / len(train_dl))
    inf_losses.append(inf_l / len(test_dl))

    # Log and save
    if tr_idx and tr_idx % 500 == 0:
        clear_output(wait=True)
        plt.plot(tr_losses, label="train loss")
        plt.plot(inf_losses, label="inf loss")
        plt.legend()
        plt.show()

        print(
            f"iteration: {tr_idx} train loss: {tr_l / len(train_dl)} inf loss: {inf_l / len(test_dl)}"
        )
        torch.save(model.state_dict(), f"sales_model_{tr_idx}_{B}.pth")

### display losses

In [ ]:
plt.plot(range(len(tr_losses)), tr_losses, label="train loss")
plt.plot(range(len(inf_losses)), inf_losses, label="test loss")
plt.legend()
plt.show()